In [97]:
import pandas as pd
import numpy as np
from snowflake.sqlalchemy import URL
from datetime import datetime
import warnings
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [98]:
observation_date = datetime(2024, 1, 31)
observation_month = "jan'24"

In [99]:
file_path = "Experian_scrub_27022024_125026PM_Niro_Disbursed_cases"
exp_acc = pd.read_csv(file_path + '/RPT_AR.txt', sep='|').drop_duplicates().reset_index(drop=True)
exp_enq = pd.read_csv(file_path + '/RPT_ENQ.txt', sep='|').drop_duplicates().reset_index(drop=True)
exp_score = pd.read_csv(file_path + '/RPT_Score.txt', sep='|').drop_duplicates().reset_index(drop=True)
exp_acct_type_map = pd.read_excel('/Users/keyurpethad/Library/CloudStorage/GoogleDrive-keyur@niro.money/My Drive/Experian/Experian Data.xlsx', sheet_name='TradeType')

In [100]:
exp_acct_type_map.head()

,ACCT_TYPE_CD,Description,Type_Trade,Security,cc,term
0,5,Credit Card,CC,Unsecured,1.0,NaN
1,47,"Instalment Loan, Automobile",AT,Secured,NaN,1.0
2,58,"Instalment Loan, Mortgage",MT,Secured,NaN,1.0
3,121,"Banking, Overdraft",PL,Unsecured,NaN,NaN
4,123,"Loan, Personal Cash",PL,Unsecured,NaN,1.0


In [101]:
print('Before dropping :', exp_acc.shape, exp_enq.shape, exp_score.shape)
exp_acc.drop_duplicates(inplace=True)
exp_enq.drop_duplicates(inplace=True)
exp_score.drop_duplicates(inplace=True)
print('After dropping :', exp_acc.shape, exp_enq.shape, exp_score.shape)

Before dropping : (1260285, 240) (440022, 5) (41988, 2)
After dropping : (1260285, 240) (440022, 5) (41988, 2)


In [102]:
exp_acc.columns = exp_acc.columns.str.lower().str.replace(' ', '_')
exp_acct_type_map.columns = exp_acct_type_map.columns.str.lower().str.replace(' ', '_')

In [103]:
exp_acc['open_dt'] = pd.to_datetime(exp_acc['open_dt'], format='%d/%m/%Y', errors = 'coerce')
exp_acc['balance_dt'] = pd.to_datetime(exp_acc['balance_dt'], format='%d/%m/%Y', errors = 'coerce')
exp_acc['closed_dt'] = pd.to_datetime(exp_acc['closed_dt'], format='%d/%m/%Y', errors = 'coerce')
exp_acc['last_payment_dt'] = pd.to_datetime(exp_acc['last_payment_dt'], format='%d/%m/%Y', errors = 'coerce')
exp_acc['write_off_status_dt'] = pd.to_datetime(exp_acc['write_off_status_dt'], format='%d/%m/%Y', errors = 'coerce')

In [104]:
exp_acc = exp_acc.merge(exp_acct_type_map, how = 'left', on='acct_type_cd')

In [105]:
exp_acc.tail()

,customer_id,acct_key,account_nb,m_sub_id,acct_type_cd,open_dt,actual_payment_am,asset_class_cd,balance_am,balance_dt,charge_off_am,closed_dt,credit_limit_am,days_past_due,dflt_status_dt,last_payment_dt,orig_loan_am,past_due_am,payment_history_grid,suit_filed_willful_dflt,written_off_and_settled_status,write_off_status_dt,responsibility_cd,portfolio_rating_type_cd,actual_payment_am_01,actual_payment_am_02,actual_payment_am_03,actual_payment_am_04,actual_payment_am_05,actual_payment_am_06,actual_payment_am_07,actual_payment_am_08,actual_payment_am_09,actual_payment_am_10,actual_payment_am_11,actual_payment_am_12,actual_payment_am_13,actual_payment_am_14,actual_payment_am_15,actual_payment_am_16,actual_payment_am_17,actual_payment_am_18,actual_payment_am_19,actual_payment_am_20,actual_payment_am_21,actual_payment_am_22,actual_payment_am_23,actual_payment_am_24,actual_payment_am_25,actual_payment_am_26,actual_payment_am_27,actual_payment_am_28,actual_payment_am_29,actual_payment_am_30,actual_payment_am_31,actual_payment_am_32,actual_payment_am_33,actual_payment_am_34,actual_payment_am_35,actual_payment_am_36,balance_am_01,balance_am_02,balance_am_03,balance_am_04,balance_am_05,balance_am_06,balance_am_07,balance_am_08,balance_am_09,balance_am_10,balance_am_11,balance_am_12,balance_am_13,balance_am_14,balance_am_15,balance_am_16,balance_am_17,balance_am_18,balance_am_19,balance_am_20,balance_am_21,balance_am_22,balance_am_23,balance_am_24,balance_am_25,balance_am_26,balance_am_27,balance_am_28,balance_am_29,balance_am_30,balance_am_31,balance_am_32,balance_am_33,balance_am_34,balance_am_35,balance_am_36,credit_limit_am_01,credit_limit_am_02,credit_limit_am_03,credit_limit_am_04,credit_limit_am_05,credit_limit_am_06,credit_limit_am_07,credit_limit_am_08,credit_limit_am_09,credit_limit_am_10,credit_limit_am_11,credit_limit_am_12,credit_limit_am_13,credit_limit_am_14,credit_limit_am_15,credit_limit_am_16,credit_limit_am_17,credit_limit_am_18,credit_limit_am_19,credit_limit_am_20,credit_limit_am_21,credit_limit_am_22,credit_limit_am_23,credit_limit_am_24,credit_limit_am_25,credit_limit_am_26,credit_limit_am_27,credit_limit_am_28,credit_limit_am_29,credit_limit_am_30,credit_limit_am_31,credit_limit_am_32,credit_limit_am_33,credit_limit_am_34,credit_limit_am_35,credit_limit_am_36,past_due_am_01,past_due_am_02,past_due_am_03,past_due_am_04,past_due_am_05,past_due_am_06,past_due_am_07,past_due_am_08,past_due_am_09,past_due_am_10,past_due_am_11,past_due_am_12,past_due_am_13,past_due_am_14,past_due_am_15,past_due_am_16,past_due_am_17,past_due_am_18,past_due_am_19,past_due_am_20,past_due_am_21,past_due_am_22,past_due_am_23,past_due_am_24,past_due_am_25,past_due_am_26,past_due_am_27,past_due_am_28,past_due_am_29,past_due_am_30,past_due_am_31,past_due_am_32,past_due_am_33,past_due_am_34,past_due_am_35,past_due_am_36,payment_rating_cd_01,payment_rating_cd_02,payment_rating_cd_03,payment_rating_cd_04,payment_rating_cd_05,payment_rating_cd_06,payment_rating_cd_07,payment_rating_cd_08,payment_rating_cd_09,payment_rating_cd_10,payment_rating_cd_11,payment_rating_cd_12,payment_rating_cd_13,payment_rating_cd_14,payment_rating_cd_15,payment_rating_cd_16,payment_rating_cd_17,payment_rating_cd_18,payment_rating_cd_19,payment_rating_cd_20,payment_rating_cd_21,payment_rating_cd_22,payment_rating_cd_23,payment_rating_cd_24,payment_rating_cd_25,payment_rating_cd_26,payment_rating_cd_27,payment_rating_cd_28,payment_rating_cd_29,payment_rating_cd_30,payment_rating_cd_31,payment_rating_cd_32,payment_rating_cd_33,payment_rating_cd_34,payment_rating_cd_35,payment_rating_cd_36,days_past_due_01,days_past_due_02,days_past_due_03,days_past_due_04,days_past_due_05,days_past_due_06,days_past_due_07,days_past_due_08,days_past_due_09,days_past_due_10,days_past_due_11,days_past_due_12,days_past_due_13,days_past_due_14,days_past_due_15,days_past_due_16,days_past_due_17,days_past_due_18,days_past_due_19,days_past_due_20,days_past_due_21,days_past_due_22,days_past_due_23,da

In [8]:
print('Before dropping :', exp_acc.shape)
exp_acc = exp_acc[(exp_acc['balance_dt'] >= datetime(2010, 1, 1)) & (exp_acc['balance_dt'] < observation_date)].reset_index(drop=True)
print('Before dropping :', exp_acc.shape)

Before dropping : (1260285, 240)
Before dropping : (968607, 240)


In [13]:
exp_acc = exp_acc.tail(5000).reset_index(drop=True)

#### Preprocessing

In [14]:
def backfill_zeros(row):
    start = None
    last_nonzero_value = None
    for i, value in enumerate(row):
        if value != 0:
            last_nonzero_value = value
            start = i
        elif last_nonzero_value is not None:
            break
        
    if start is not None:
        row[start+1:-1] = row[-2]
    return row

In [15]:
# Filling null/0 credit limit with back_fill followed by orig_loan_am
exp_acc.loc[:, [x for x in exp_acc.columns if 'credit_limit_am_' in x]] = exp_acc[[x for x in exp_acc.columns if 'credit_limit_am_' in x]].fillna(0).apply(backfill_zeros, axis=1)
exp_acc.loc[:, [x for x in exp_acc.columns if 'credit_limit_am_' in x]] = np.where((exp_acc.loc[:, [x for x in exp_acc.columns if 'credit_limit_am_' in x]].isna() | exp_acc.loc[:, [x for x in exp_acc.columns if 'credit_limit_am_' in x]] == 0), pd.concat([exp_acc['orig_loan_am'] for _ in range(36)], axis=1), exp_acc.loc[:, [x for x in exp_acc.columns if 'credit_limit_am_' in x]])

#Filling null balance with 0
exp_acc.loc[:, [x for x in exp_acc.columns if 'balance_am_' in x]] = exp_acc[[x for x in exp_acc.columns if 'balance_am_' in x]].fillna(0)

In [16]:
for j in range(len(exp_acc)):
    for i in range(36):
        if i+1 < 10:
            x = '0'+str(i+1)
        else: x = str(i+1)
        exp_acc.loc[j, 'actual_payment_' + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).year) + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).month)] = exp_acc.loc[j, 'actual_payment_am_' + x]
        exp_acc.loc[j, 'balance_' + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).year) + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).month)] = exp_acc.loc[j, 'balance_am_' + x]
        exp_acc.loc[j, 'credit_limit_' + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).year) + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).month)] = exp_acc.loc[j, 'credit_limit_am_' + x]
        exp_acc.loc[j, 'past_due_' + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).year) + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).month)] = exp_acc.loc[j, 'past_due_am_' + x]
        exp_acc.loc[j, 'payment_rating_' + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).year) + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).month)] = exp_acc.loc[j, 'payment_rating_cd_' + x]
        exp_acc.loc[j, 'days_past_due_' + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).year) + str((exp_acc.loc[j,'balance_dt'] - pd.DateOffset(months=i)).month)] = exp_acc.loc[j, 'days_past_due_' + x]

In [17]:
exp_agg = pd.DataFrame(exp_acc['customer_id'].drop_duplicates())

In [18]:
exp_acc.head()

,customer_id,acct_key,account_nb,m_sub_id,acct_type_cd,open_dt,actual_payment_am,asset_class_cd,balance_am,balance_dt,charge_off_am,closed_dt,credit_limit_am,days_past_due,dflt_status_dt,last_payment_dt,orig_loan_am,past_due_am,payment_history_grid,suit_filed_willful_dflt,written_off_and_settled_status,write_off_status_dt,responsibility_cd,portfolio_rating_type_cd,actual_payment_am_01,actual_payment_am_02,actual_payment_am_03,actual_payment_am_04,actual_payment_am_05,actual_payment_am_06,actual_payment_am_07,actual_payment_am_08,actual_payment_am_09,actual_payment_am_10,actual_payment_am_11,actual_payment_am_12,actual_payment_am_13,actual_payment_am_14,actual_payment_am_15,actual_payment_am_16,actual_payment_am_17,actual_payment_am_18,actual_payment_am_19,actual_payment_am_20,actual_payment_am_21,actual_payment_am_22,actual_payment_am_23,actual_payment_am_24,actual_payment_am_25,actual_payment_am_26,actual_payment_am_27,actual_payment_am_28,actual_payment_am_29,actual_payment_am_30,actual_payment_am_31,actual_payment_am_32,actual_payment_am_33,actual_payment_am_34,actual_payment_am_35,actual_payment_am_36,balance_am_01,balance_am_02,balance_am_03,balance_am_04,balance_am_05,balance_am_06,balance_am_07,balance_am_08,balance_am_09,balance_am_10,balance_am_11,balance_am_12,balance_am_13,balance_am_14,balance_am_15,balance_am_16,balance_am_17,balance_am_18,balance_am_19,balance_am_20,balance_am_21,balance_am_22,balance_am_23,balance_am_24,balance_am_25,balance_am_26,balance_am_27,balance_am_28,balance_am_29,balance_am_30,balance_am_31,balance_am_32,balance_am_33,balance_am_34,balance_am_35,balance_am_36,credit_limit_am_01,credit_limit_am_02,credit_limit_am_03,credit_limit_am_04,credit_limit_am_05,credit_limit_am_06,credit_limit_am_07,credit_limit_am_08,credit_limit_am_09,credit_limit_am_10,credit_limit_am_11,credit_limit_am_12,credit_limit_am_13,credit_limit_am_14,credit_limit_am_15,credit_limit_am_16,credit_limit_am_17,credit_limit_am_18,credit_limit_am_19,credit_limit_am_20,credit_limit_am_21,credit_limit_am_22,credit_limit_am_23,credit_limit_am_24,credit_limit_am_25,credit_limit_am_26,credit_limit_am_27,credit_limit_am_28,credit_limit_am_29,credit_limit_am_30,credit_limit_am_31,credit_limit_am_32,credit_limit_am_33,credit_limit_am_34,credit_limit_am_35,credit_limit_am_36,past_due_am_01,past_due_am_02,past_due_am_03,past_due_am_04,past_due_am_05,past_due_am_06,past_due_am_07,past_due_am_08,past_due_am_09,past_due_am_10,past_due_am_11,past_due_am_12,past_due_am_13,past_due_am_14,past_due_am_15,past_due_am_16,past_due_am_17,past_due_am_18,past_due_am_19,past_due_am_20,past_due_am_21,past_due_am_22,past_due_am_23,past_due_am_24,past_due_am_25,past_due_am_26,past_due_am_27,past_due_am_28,past_due_am_29,past_due_am_30,past_due_am_31,past_due_am_32,past_due_am_33,past_due_am_34,past_due_am_35,past_due_am_36,payment_rating_cd_01,payment_rating_cd_02,payment_rating_cd_03,payment_rating_cd_04,payment_rating_cd_05,payment_rating_cd_06,payment_rating_cd_07,payment_rating_cd_08,payment_rating_cd_09,payment_rating_cd_10,payment_rating_cd_11,payment_rating_cd_12,payment_rating_cd_13,payment_rating_cd_14,payment_rating_cd_15,payment_rating_cd_16,payment_rating_cd_17,payment_rating_cd_18,payment_rating_cd_19,payment_rating_cd_20,payment_rating_cd_21,payment_rating_cd_22,payment_rating_cd_23,payment_rating_cd_24,payment_rating_cd_25,payment_rating_cd_26,payment_rating_cd_27,payment_rating_cd_28,payment_rating_cd_29,payment_rating_cd_30,payment_rating_cd_31,payment_rating_cd_32,payment_rating_cd_33,payment_rating_cd_34,payment_rating_cd_35,payment_rating_cd_36,days_past_due_01,days_past_due_02,days_past_due_03,days_past_due_04,days_past_due_05,days_past_due_06,days_past_due_07,days_past_due_08,days_past_due_09,days_past_due_10,days_past_due_11,days_past_due_12,days_past_due_13,days_past_due_14,days_past_due_15,days_past_due_16,days_past_due_17,days_past_due_18,days_past_due_19,days_past_due_20,days_past_due_21,days_past_due_22,days_past_due_23,da

##### 1. Written_off, Settled, suit_filed, wilful_default, restructered, acc_purchased/sold, clear_existing_status

In [19]:
for x in [3,12,24]: 
    exp_data = exp_acc[exp_acc['balance_dt'] >= observation_date - pd.DateOffset(months=x)].reset_index(drop=True)
    exp_data['suit_filed_willful_dflt_l' + str(x) + 'm'] = np.where(~exp_data['suit_filed_willful_dflt'].isin([0,200]) & exp_data['suit_filed_willful_dflt'].notna(), 1, 0)
    exp_data['written_off_and_settled_status_l' + str(x) + 'm'] = np.where(exp_data['written_off_and_settled_status'].isin([2,3,4,6,8,9]) & exp_data['written_off_and_settled_status'].notna(), 1, 0)
    exp_data['acc_purchased_sold_l' + str(x) + 'm'] = np.where(exp_data['written_off_and_settled_status'].isin([5,6,7,8,9,10]) & exp_data['written_off_and_settled_status'].notna(), 1, 0)
    exp_data['restructured_l' + str(x) + 'm'] = np.where(exp_data['suit_filed_willful_dflt'].isin([200]) | (exp_data['written_off_and_settled_status'].isin([0,1,11,12]) & exp_data['written_off_and_settled_status'].notna()), 1, 0)
    exp_data['clear_existing_status_l' + str(x) + 'm'] = np.where(exp_data['written_off_and_settled_status'].isin([99]) & exp_data['written_off_and_settled_status'].notna(), 1, 0)
    exp_data['any_derog_l' + str(x) + 'm'] = np.where((exp_data['suit_filed_willful_dflt_l' + str(x) + 'm'] | exp_data['written_off_and_settled_status_l' + str(x) + 'm'] | exp_data['acc_purchased_sold_l' + str(x) + 'm'] | exp_data['restructured_l' + str(x) + 'm'] | exp_data['clear_existing_status_l' + str(x) + 'm']), 1, 0)
    exp_data['LA_suit_filed_willful_dflt_l' + str(x) + 'm'] = exp_data['suit_filed_willful_dflt_l' + str(x) + 'm'] * exp_data['orig_loan_am']
    exp_data['LA_written_off_and_settled_status_l' + str(x) + 'm'] = exp_data['written_off_and_settled_status_l' + str(x) + 'm'] * exp_data['orig_loan_am']
    exp_data['LA_acc_purchased_sold_l' + str(x) + 'm'] = exp_data['acc_purchased_sold_l' + str(x) + 'm'] * exp_data['orig_loan_am']
    exp_data['LA_restructured_l' + str(x) + 'm'] = exp_data['restructured_l' + str(x) + 'm'] * exp_data['orig_loan_am']
    exp_data['LA_clear_existing_status_l' + str(x) + 'm'] = exp_data['clear_existing_status_l' + str(x) + 'm'] * exp_data['orig_loan_am']
    exp_data['LA_any_derog_l' + str(x) + 'm'] = exp_data['any_derog_l' + str(x) + 'm'] * exp_data['orig_loan_am']
    exp_data['balance_suit_filed_willful_dflt_l' + str(x) + 'm'] = exp_data['suit_filed_willful_dflt_l' + str(x) + 'm'] * exp_data['balance_am']
    exp_data['balance_written_off_and_settled_status_l' + str(x) + 'm'] = exp_data['written_off_and_settled_status_l' + str(x) + 'm'] * exp_data['balance_am']
    exp_data['balance_acc_purchased_sold_l' + str(x) + 'm'] = exp_data['acc_purchased_sold_l' + str(x) + 'm'] * exp_data['balance_am']
    exp_data['balance_restructured_l' + str(x) + 'm'] = exp_data['restructured_l' + str(x) + 'm'] * exp_data['balance_am']
    exp_data['balance_clear_existing_status_l' + str(x) + 'm'] = exp_data['clear_existing_status_l' + str(x) + 'm'] * exp_data['balance_am']
    exp_data['balance_any_derog_l' + str(x) + 'm'] = exp_data['any_derog_l' + str(x) + 'm'] * exp_data['balance_am']
    exp_agg = exp_agg.merge(exp_data.groupby('customer_id')[['suit_filed_willful_dflt_l' + str(x) + 'm', 'written_off_and_settled_status_l' + str(x) + 'm', 'acc_purchased_sold_l' + str(x) + 'm', 'restructured_l' + str(x) + 'm', 'clear_existing_status_l' + str(x) + 'm', 'any_derog_l' + str(x) + 'm',
                                                             'LA_suit_filed_willful_dflt_l' + str(x) + 'm', 'LA_written_off_and_settled_status_l' + str(x) + 'm', 'LA_acc_purchased_sold_l' + str(x) + 'm', 'LA_restructured_l' + str(x) + 'm', 'LA_clear_existing_status_l' + str(x) + 'm', 'LA_any_derog_l' + str(x) + 'm',
                                                             'balance_suit_filed_willful_dflt_l' + str(x) + 'm', 'balance_written_off_and_settled_status_l' + str(x) + 'm', 'balance_acc_purchased_sold_l' + str(x) + 'm', 'balance_restructured_l' + str(x) + 'm', 'balance_clear_existing_status_l' + str(x) + 'm', 'balance_any_derog_l' + str(x) + 'm',]].sum(), on = 'customer_id', how = 'left')

exp_acc['suit_filed_willful_dflt_ever'] = np.where(~exp_acc['suit_filed_willful_dflt'].isin([0,200]) & exp_acc['suit_filed_willful_dflt'].notna(), 1, 0)
exp_acc['written_off_and_settled_status_ever'] = np.where(exp_acc['written_off_and_settled_status'].isin([2,3,4,6,8,9]) & exp_acc['written_off_and_settled_status'].notna(), 1, 0)
exp_acc['acc_purchased_sold_ever'] = np.where(exp_acc['written_off_and_settled_status'].isin([5,6,7,8,9,10]) & exp_acc['written_off_and_settled_status'].notna(), 1, 0)
exp_acc['restructured_ever'] = np.where(exp_acc['suit_filed_willful_dflt'].isin([200]) | (exp_acc['written_off_and_settled_status'].isin([0,1,11,12]) & exp_acc['written_off_and_settled_status'].notna()), 1, 0)
exp_acc['clear_existing_status_ever'] = np.where(exp_acc['written_off_and_settled_status'].isin([99]) & exp_acc['written_off_and_settled_status'].notna(), 1, 0)
exp_acc['any_derog_ever'] = np.where((exp_acc['suit_filed_willful_dflt_ever'] | exp_acc['written_off_and_settled_status_ever'] | exp_acc['acc_purchased_sold_ever'] | exp_acc['restructured_ever'] | exp_acc['clear_existing_status_ever']), 1, 0)
exp_acc['LA_suit_filed_willful_dflt_ever'] = exp_acc['suit_filed_willful_dflt_ever'] * exp_acc['orig_loan_am']
exp_acc['LA_written_off_and_settled_status_ever'] = exp_acc['written_off_and_settled_status_ever'] * exp_acc['orig_loan_am']
exp_acc['LA_acc_purchased_sold_ever'] = exp_acc['acc_purchased_sold_ever'] * exp_acc['orig_loan_am']
exp_acc['LA_restructured_ever'] = exp_acc['restructured_ever'] * exp_acc['orig_loan_am']
exp_acc['LA_clear_existing_status_ever'] = exp_acc['clear_existing_status_ever'] * exp_acc['orig_loan_am']
exp_acc['LA_any_derog_ever'] = exp_acc['any_derog_ever'] * exp_acc['orig_loan_am']
exp_acc['balance_suit_filed_willful_dflt_ever'] = exp_acc['suit_filed_willful_dflt_ever'] * exp_acc['balance_am']
exp_acc['balance_written_off_and_settled_status_ever'] = exp_acc['written_off_and_settled_status_ever'] * exp_acc['balance_am']
exp_acc['balance_acc_purchased_sold_ever'] = exp_acc['acc_purchased_sold_ever'] * exp_acc['balance_am']
exp_acc['balance_restructured_ever'] = exp_acc['restructured_ever'] * exp_acc['balance_am']
exp_acc['balance_clear_existing_status_ever'] = exp_acc['clear_existing_status_ever'] * exp_acc['balance_am']
exp_acc['balance_any_derog_ever'] = exp_acc['any_derog_ever'] * exp_acc['balance_am']
exp_agg = exp_agg.merge(exp_acc.groupby('customer_id')[['suit_filed_willful_dflt_ever', 'written_off_and_settled_status_ever', 'acc_purchased_sold_ever', 'restructured_ever', 'clear_existing_status_ever', 'any_derog_ever',
                                                        'LA_suit_filed_willful_dflt_ever', 'LA_written_off_and_settled_status_ever', 'LA_acc_purchased_sold_ever', 'LA_restructured_ever', 'LA_clear_existing_status_ever', 'LA_any_derog_ever',
                                                        'balance_suit_filed_willful_dflt_ever', 'balance_written_off_and_settled_status_ever', 'balance_acc_purchased_sold_ever', 'balance_restructured_ever', 'balance_clear_existing_status_ever', 'balance_any_derog_ever',]].sum(), on = 'customer_id', how = 'left')


In [20]:
exp_agg.head()

,customer_id,suit_filed_willful_dflt_l3m,written_off_and_settled_status_l3m,acc_purchased_sold_l3m,restructured_l3m,clear_existing_status_l3m,any_derog_l3m,LA_suit_filed_willful_dflt_l3m,LA_written_off_and_settled_status_l3m,LA_acc_purchased_sold_l3m,LA_restructured_l3m,LA_clear_existing_status_l3m,LA_any_derog_l3m,balance_suit_filed_willful_dflt_l3m,balance_written_off_and_settled_status_l3m,balance_acc_purchased_sold_l3m,balance_restructured_l3m,balance_clear_existing_status_l3m,balance_any_derog_l3m,suit_filed_willful_dflt_l12m,written_off_and_settled_status_l12m,acc_purchased_sold_l12m,restructured_l12m,clear_existing_status_l12m,any_derog_l12m,LA_suit_filed_willful_dflt_l12m,LA_written_off_and_settled_status_l12m,LA_acc_purchased_sold_l12m,LA_restructured_l12m,LA_clear_existing_status_l12m,LA_any_derog_l12m,balance_suit_filed_willful_dflt_l12m,balance_written_off_and_settled_status_l12m,balance_acc_purchased_sold_l12m,balance_restructured_l12m,balance_clear_existing_status_l12m,balance_any_derog_l12m,suit_filed_willful_dflt_l24m,written_off_and_settled_status_l24m,acc_purchased_sold_l24m,restructured_l24m,clear_existing_status_l24m,any_derog_l24m,LA_suit_filed_willful_dflt_l24m,LA_written_off_and_settled_status_l24m,LA_acc_purchased_sold_l24m,LA_restructured_l24m,LA_clear_existing_status_l24m,LA_any_derog_l24m,balance_suit_filed_willful_dflt_l24m,balance_written_off_and_settled_status_l24m,balance_acc_purchased_sold_l24m,balance_restructured_l24m,balance_clear_existing_status_l24m,balance_any_derog_l24m,suit_filed_willful_dflt_ever,written_off_and_settled_status_ever,acc_purchased_sold_ever,restructured_ever,clear_existing_status_ever,any_derog_ever,LA_suit_filed_willful_dflt_ever,LA_written_off_and_settled_status_ever,LA_acc_purchased_sold_ever,LA_restructured_ever,LA_clear_existing_status_ever,LA_any_derog_ever,balance_suit_filed_willful_dflt_ever,balance_written_off_and_settled_status_ever,balance_acc_purchased_sold_ever,balance_restructured_ever,balance_clear_existing_status_ever,balance_any_derog_ever
0,7042158364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9071353283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9895490565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8420743749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8508831665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
exp_acc.head()

,customer_id,acct_key,account_nb,m_sub_id,acct_type_cd,open_dt,actual_payment_am,asset_class_cd,balance_am,balance_dt,charge_off_am,closed_dt,credit_limit_am,days_past_due,dflt_status_dt,last_payment_dt,orig_loan_am,past_due_am,payment_history_grid,suit_filed_willful_dflt,written_off_and_settled_status,write_off_status_dt,responsibility_cd,portfolio_rating_type_cd,actual_payment_am_01,actual_payment_am_02,actual_payment_am_03,actual_payment_am_04,actual_payment_am_05,actual_payment_am_06,actual_payment_am_07,actual_payment_am_08,actual_payment_am_09,actual_payment_am_10,actual_payment_am_11,actual_payment_am_12,actual_payment_am_13,actual_payment_am_14,actual_payment_am_15,actual_payment_am_16,actual_payment_am_17,actual_payment_am_18,actual_payment_am_19,actual_payment_am_20,actual_payment_am_21,actual_payment_am_22,actual_payment_am_23,actual_payment_am_24,actual_payment_am_25,actual_payment_am_26,actual_payment_am_27,actual_payment_am_28,actual_payment_am_29,actual_payment_am_30,actual_payment_am_31,actual_payment_am_32,actual_payment_am_33,actual_payment_am_34,actual_payment_am_35,actual_payment_am_36,balance_am_01,balance_am_02,balance_am_03,balance_am_04,balance_am_05,balance_am_06,balance_am_07,balance_am_08,balance_am_09,balance_am_10,balance_am_11,balance_am_12,balance_am_13,balance_am_14,balance_am_15,balance_am_16,balance_am_17,balance_am_18,balance_am_19,balance_am_20,balance_am_21,balance_am_22,balance_am_23,balance_am_24,balance_am_25,balance_am_26,balance_am_27,balance_am_28,balance_am_29,balance_am_30,balance_am_31,balance_am_32,balance_am_33,balance_am_34,balance_am_35,balance_am_36,credit_limit_am_01,credit_limit_am_02,credit_limit_am_03,credit_limit_am_04,credit_limit_am_05,credit_limit_am_06,credit_limit_am_07,credit_limit_am_08,credit_limit_am_09,credit_limit_am_10,credit_limit_am_11,credit_limit_am_12,credit_limit_am_13,credit_limit_am_14,credit_limit_am_15,credit_limit_am_16,credit_limit_am_17,credit_limit_am_18,credit_limit_am_19,credit_limit_am_20,credit_limit_am_21,credit_limit_am_22,credit_limit_am_23,credit_limit_am_24,credit_limit_am_25,credit_limit_am_26,credit_limit_am_27,credit_limit_am_28,credit_limit_am_29,credit_limit_am_30,credit_limit_am_31,credit_limit_am_32,credit_limit_am_33,credit_limit_am_34,credit_limit_am_35,credit_limit_am_36,past_due_am_01,past_due_am_02,past_due_am_03,past_due_am_04,past_due_am_05,past_due_am_06,past_due_am_07,past_due_am_08,past_due_am_09,past_due_am_10,past_due_am_11,past_due_am_12,past_due_am_13,past_due_am_14,past_due_am_15,past_due_am_16,past_due_am_17,past_due_am_18,past_due_am_19,past_due_am_20,past_due_am_21,past_due_am_22,past_due_am_23,past_due_am_24,past_due_am_25,past_due_am_26,past_due_am_27,past_due_am_28,past_due_am_29,past_due_am_30,past_due_am_31,past_due_am_32,past_due_am_33,past_due_am_34,past_due_am_35,past_due_am_36,payment_rating_cd_01,payment_rating_cd_02,payment_rating_cd_03,payment_rating_cd_04,payment_rating_cd_05,payment_rating_cd_06,payment_rating_cd_07,payment_rating_cd_08,payment_rating_cd_09,payment_rating_cd_10,payment_rating_cd_11,payment_rating_cd_12,payment_rating_cd_13,payment_rating_cd_14,payment_rating_cd_15,payment_rating_cd_16,payment_rating_cd_17,payment_rating_cd_18,payment_rating_cd_19,payment_rating_cd_20,payment_rating_cd_21,payment_rating_cd_22,payment_rating_cd_23,payment_rating_cd_24,payment_rating_cd_25,payment_rating_cd_26,payment_rating_cd_27,payment_rating_cd_28,payment_rating_cd_29,payment_rating_cd_30,payment_rating_cd_31,payment_rating_cd_32,payment_rating_cd_33,payment_rating_cd_34,payment_rating_cd_35,payment_rating_cd_36,days_past_due_01,days_past_due_02,days_past_due_03,days_past_due_04,days_past_due_05,days_past_due_06,days_past_due_07,days_past_due_08,days_past_due_09,days_past_due_10,days_past_due_11,days_past_due_12,days_past_due_13,days_past_due_14,days_past_due_15,days_past_due_16,days_past_due_17,days_past_due_18,days_past_due_19,days_past_due_20,days_past_due_21,days_past_due_22,days_past_due_23,da

In [22]:
months = [x.split('_')[-1] for x in exp_acc.columns if 'credit_limit_' in x and 'credit_limit_am' not in x]

##### cc_util

In [23]:
exp_cc = exp_acc[exp_acc.type_trade == 'CC'].reset_index(drop=True)

In [24]:
exp_cc = exp_cc.groupby('customer_id')[['balance_' + x for x in months] + ['credit_limit_' + x for x in months]].sum().reset_index()

In [25]:
for x in months:
    exp_cc['cc_util_' + x] = exp_cc['balance_' + x] / exp_cc['credit_limit_' + x]

In [26]:
exp_cc.head()

,customer_id,balance_20215,balance_20214,balance_20213,balance_20212,balance_20211,balance_202012,balance_202011,balance_202010,balance_20209,balance_20208,balance_20207,balance_20206,balance_20205,balance_20204,balance_20203,balance_20202,balance_20201,balance_201912,balance_201911,balance_201910,balance_20199,balance_20198,balance_20197,balance_20196,balance_20195,balance_20194,balance_20193,balance_20192,balance_20191,balance_201812,balance_201811,balance_201810,balance_20189,balance_20188,balance_20187,balance_20186,balance_20231,balance_202212,balance_202211,balance_202210,balance_20229,balance_20228,balance_20227,balance_20226,balance_20225,balance_20224,balance_20223,balance_20222,balance_20221,balance_202112,balance_202111,balance_202110,balance_20219,balance_20218,balance_20217,balance_20216,balance_20185,balance_20184,balance_20183,balance_20182,balance_20181,balance_201712,balance_201711,balance_201710,balance_20179,balance_20178,balance_20177,balance_20176,balance_20175,balance_20174,balance_20173,balance_20172,balance_20171,balance_201612,balance_201611,balance_201610,balance_20169,balance_20168,balance_20167,balance_20166,balance_20165,balance_20164,balance_20163,balance_20162,balance_20161,balance_201512,balance_202312,balance_202311,balance_202310,balance_20239,balance_20238,balance_20237,balance_20236,balance_20235,balance_20234,balance_20233,balance_20232,balance_20241,balance_201511,balance_201510,balance_20159,balance_20158,balance_20157,balance_20156,balance_20155,balance_20154,balance_20153,balance_20152,balance_20151,balance_201412,balance_201411,balance_201410,balance_20149,balance_20148,balance_20147,balance_20146,balance_20145,balance_20144,balance_20143,balance_20142,balance_20141,balance_201312,balance_201311,balance_201310,balance_20139,balance_20138,balance_20137,balance_20136,balance_20135,balance_20134,balance_20133,balance_20132,balance_20131,balance_201212,balance_201211,balance_201210,balance_20129,balance_20128,balance_20127,balance_20126,balance_20125,balance_20124,balance_20123,balance_20122,balance_20121,balance_201112,balance_201111,balance_201110,balance_20119,balance_20118,balance_20117,balance_20116,balance_20115,balance_20114,balance_20113,balance_20112,balance_20111,balance_201012,balance_201011,balance_201010,balance_20109,balance_20108,balance_20107,balance_20106,balance_20105,balance_20104,balance_20103,balance_20102,balance_20101,balance_200912,balance_200911,balance_200910,balance_20099,balance_20098,balance_20097,balance_20096,balance_20095,balance_20094,balance_20093,balance_20092,balance_20091,balance_200812,balance_200811,balance_200810,balance_20089,balance_20088,balance_20087,balance_20086,balance_20085,balance_20084,balance_20083,balance_20082,balance_20081,balance_200712,balance_200711,balance_200710,balance_20079,balance_20078,credit_limit_20215,credit_limit_20214,credit_limit_20213,credit_limit_20212,credit_limit_20211,credit_limit_202012,credit_limit_202011,credit_limit_202010,credit_limit_20209,credit_limit_20208,credit_limit_20207,credit_limit_20206,credit_limit_20205,credit_limit_20204,credit_limit_20203,credit_limit_20202,credit_limit_20201,credit_limit_201912,credit_limit_201911,credit_limit_201910,credit_limit_20199,credit_limit_20198,credit_limit_20197,credit_limit_20196,credit_limit_20195,credit_limit_20194,credit_limit_20193,credit_limit_20192,credit_limit_20191,credit_limit_201812,credit_limit_201811,credit_limit_201810,credit_limit_20189,credit_limit_20188,credit_limit_20187,credit_limit_20186,credit_limit_20231,credit_limit_202212,credit_limit_202211,credit_limit_202210,credit_limit_20229,credit_limit_20228,credit_limit_20227,credit_limit_20226,credit_limit_20225,credit_limit_20224,credit_limit_20223,credit_limit_20222,credit_limit_20221,credit_limit_202112,credit_limit_202111,credit_limit_202110,credit_limit_20219,credit_limit_20218,credit_limit_20217,credit_limit_20216,credit_limit_20185,credit_limit_20184,credit_limit_20183,credit_limit_20182,credit_li

In [34]:
for i in [3,6,12]:
    req_months = []
    for j in range(1,i+1):
        req_months.append(str((observation_date - pd.DateOffset(months=j)).year) + str((observation_date - pd.DateOffset(months=j)).month))
    print(req_months)

['202312', '202311', '202310']
['202312', '202311', '202310', '20239', '20238', '20237']
['202312', '202311', '202310', '20239', '20238', '20237', '20236', '20235', '20234', '20233', '20232', '20231']


In [43]:
for i in [3,6,12]:
    req_months = [(str((observation_date - pd.DateOffset(months=j)).year) + str((observation_date - pd.DateOffset(months=j)).month)) for j in range(1,i+1)]
    exp_cc['cc_util_l' + str(i) + 'm'] = exp_cc[['cc_util_' + x for x in req_months]].mean(axis=1)

In [45]:
exp_cc['cc_util_l3m_by_cc_util_l6m'] = exp_cc['cc_util_l3m'] / exp_cc['cc_util_l6m']

In [46]:
exp_cc.head()

,customer_id,balance_20215,balance_20214,balance_20213,balance_20212,balance_20211,balance_202012,balance_202011,balance_202010,balance_20209,balance_20208,balance_20207,balance_20206,balance_20205,balance_20204,balance_20203,balance_20202,balance_20201,balance_201912,balance_201911,balance_201910,balance_20199,balance_20198,balance_20197,balance_20196,balance_20195,balance_20194,balance_20193,balance_20192,balance_20191,balance_201812,balance_201811,balance_201810,balance_20189,balance_20188,balance_20187,balance_20186,balance_20231,balance_202212,balance_202211,balance_202210,balance_20229,balance_20228,balance_20227,balance_20226,balance_20225,balance_20224,balance_20223,balance_20222,balance_20221,balance_202112,balance_202111,balance_202110,balance_20219,balance_20218,balance_20217,balance_20216,balance_20185,balance_20184,balance_20183,balance_20182,balance_20181,balance_201712,balance_201711,balance_201710,balance_20179,balance_20178,balance_20177,balance_20176,balance_20175,balance_20174,balance_20173,balance_20172,balance_20171,balance_201612,balance_201611,balance_201610,balance_20169,balance_20168,balance_20167,balance_20166,balance_20165,balance_20164,balance_20163,balance_20162,balance_20161,balance_201512,balance_202312,balance_202311,balance_202310,balance_20239,balance_20238,balance_20237,balance_20236,balance_20235,balance_20234,balance_20233,balance_20232,balance_20241,balance_201511,balance_201510,balance_20159,balance_20158,balance_20157,balance_20156,balance_20155,balance_20154,balance_20153,balance_20152,balance_20151,balance_201412,balance_201411,balance_201410,balance_20149,balance_20148,balance_20147,balance_20146,balance_20145,balance_20144,balance_20143,balance_20142,balance_20141,balance_201312,balance_201311,balance_201310,balance_20139,balance_20138,balance_20137,balance_20136,balance_20135,balance_20134,balance_20133,balance_20132,balance_20131,balance_201212,balance_201211,balance_201210,balance_20129,balance_20128,balance_20127,balance_20126,balance_20125,balance_20124,balance_20123,balance_20122,balance_20121,balance_201112,balance_201111,balance_201110,balance_20119,balance_20118,balance_20117,balance_20116,balance_20115,balance_20114,balance_20113,balance_20112,balance_20111,balance_201012,balance_201011,balance_201010,balance_20109,balance_20108,balance_20107,balance_20106,balance_20105,balance_20104,balance_20103,balance_20102,balance_20101,balance_200912,balance_200911,balance_200910,balance_20099,balance_20098,balance_20097,balance_20096,balance_20095,balance_20094,balance_20093,balance_20092,balance_20091,balance_200812,balance_200811,balance_200810,balance_20089,balance_20088,balance_20087,balance_20086,balance_20085,balance_20084,balance_20083,balance_20082,balance_20081,balance_200712,balance_200711,balance_200710,balance_20079,balance_20078,credit_limit_20215,credit_limit_20214,credit_limit_20213,credit_limit_20212,credit_limit_20211,credit_limit_202012,credit_limit_202011,credit_limit_202010,credit_limit_20209,credit_limit_20208,credit_limit_20207,credit_limit_20206,credit_limit_20205,credit_limit_20204,credit_limit_20203,credit_limit_20202,credit_limit_20201,credit_limit_201912,credit_limit_201911,credit_limit_201910,credit_limit_20199,credit_limit_20198,credit_limit_20197,credit_limit_20196,credit_limit_20195,credit_limit_20194,credit_limit_20193,credit_limit_20192,credit_limit_20191,credit_limit_201812,credit_limit_201811,credit_limit_201810,credit_limit_20189,credit_limit_20188,credit_limit_20187,credit_limit_20186,credit_limit_20231,credit_limit_202212,credit_limit_202211,credit_limit_202210,credit_limit_20229,credit_limit_20228,credit_limit_20227,credit_limit_20226,credit_limit_20225,credit_limit_20224,credit_limit_20223,credit_limit_20222,credit_limit_20221,credit_limit_202112,credit_limit_202111,credit_limit_202110,credit_limit_20219,credit_limit_20218,credit_limit_20217,credit_limit_20216,credit_limit_20185,credit_limit_20184,credit_limit_20183,credit_limit_20182,credit_li

##### Decrease in payment amount

In [47]:
exp_pay = exp_acc.groupby('customer_id')[['actual_payment_' + x for x in months]].sum().reset_index()

In [50]:
for i in [3,6,12]:
    req_months = [(str((observation_date - pd.DateOffset(months=j)).year) + str((observation_date - pd.DateOffset(months=j)).month)) for j in range(1,i+1)]
    exp_pay['actual_payment_l' + str(i) + 'm'] = exp_pay[['actual_payment_' + x for x in req_months]].mean(axis=1)

In [51]:
exp_pay['actual_payment_l3m_by_actual_payment_l6m'] = exp_pay['actual_payment_l3m'] / exp_pay['actual_payment_l6m']
exp_pay['actual_payment_l6m_by_actual_payment_l12m'] = exp_pay['actual_payment_l6m'] / exp_pay['actual_payment_l12m']

In [52]:
exp_pay.head()

,customer_id,actual_payment_20215,actual_payment_20214,actual_payment_20213,actual_payment_20212,actual_payment_20211,actual_payment_202012,actual_payment_202011,actual_payment_202010,actual_payment_20209,actual_payment_20208,actual_payment_20207,actual_payment_20206,actual_payment_20205,actual_payment_20204,actual_payment_20203,actual_payment_20202,actual_payment_20201,actual_payment_201912,actual_payment_201911,actual_payment_201910,actual_payment_20199,actual_payment_20198,actual_payment_20197,actual_payment_20196,actual_payment_20195,actual_payment_20194,actual_payment_20193,actual_payment_20192,actual_payment_20191,actual_payment_201812,actual_payment_201811,actual_payment_201810,actual_payment_20189,actual_payment_20188,actual_payment_20187,actual_payment_20186,actual_payment_20231,actual_payment_202212,actual_payment_202211,actual_payment_202210,actual_payment_20229,actual_payment_20228,actual_payment_20227,actual_payment_20226,actual_payment_20225,actual_payment_20224,actual_payment_20223,actual_payment_20222,actual_payment_20221,actual_payment_202112,actual_payment_202111,actual_payment_202110,actual_payment_20219,actual_payment_20218,actual_payment_20217,actual_payment_20216,actual_payment_20185,actual_payment_20184,actual_payment_20183,actual_payment_20182,actual_payment_20181,actual_payment_201712,actual_payment_201711,actual_payment_201710,actual_payment_20179,actual_payment_20178,actual_payment_20177,actual_payment_20176,actual_payment_20175,actual_payment_20174,actual_payment_20173,actual_payment_20172,actual_payment_20171,actual_payment_201612,actual_payment_201611,actual_payment_201610,actual_payment_20169,actual_payment_20168,actual_payment_20167,actual_payment_20166,actual_payment_20165,actual_payment_20164,actual_payment_20163,actual_payment_20162,actual_payment_20161,actual_payment_201512,actual_payment_202312,actual_payment_202311,actual_payment_202310,actual_payment_20239,actual_payment_20238,actual_payment_20237,actual_payment_20236,actual_payment_20235,actual_payment_20234,actual_payment_20233,actual_payment_20232,actual_payment_20241,actual_payment_201511,actual_payment_201510,actual_payment_20159,actual_payment_20158,actual_payment_20157,actual_payment_20156,actual_payment_20155,actual_payment_20154,actual_payment_20153,actual_payment_20152,actual_payment_20151,actual_payment_201412,actual_payment_201411,actual_payment_201410,actual_payment_20149,actual_payment_20148,actual_payment_20147,actual_payment_20146,actual_payment_20145,actual_payment_20144,actual_payment_20143,actual_payment_20142,actual_payment_20141,actual_payment_201312,actual_payment_201311,actual_payment_201310,actual_payment_20139,actual_payment_20138,actual_payment_20137,actual_payment_20136,actual_payment_20135,actual_payment_20134,actual_payment_20133,actual_payment_20132,actual_payment_20131,actual_payment_201212,actual_payment_201211,actual_payment_201210,actual_payment_20129,actual_payment_20128,actual_payment_20127,actual_payment_20126,actual_payment_20125,actual_payment_20124,actual_payment_20123,actual_payment_20122,actual_payment_20121,actual_payment_201112,actual_payment_201111,actual_payment_201110,actual_payment_20119,actual_payment_20118,actual_payment_20117,actual_payment_20116,actual_payment_20115,actual_payment_20114,actual_payment_20113,actual_payment_20112,actual_payment_20111,actual_payment_201012,actual_payment_201011,actual_payment_201010,actual_payment_20109,actual_payment_20108,actual_payment_20107,actual_payment_20106,actual_payment_20105,actual_payment_20104,actual_payment_20103,actual_payment_20102,actual_payment_20101,actual_payment_200912,actual_payment_200911,actual_payment_200910,actual_payment_20099,actual_payment_20098,actual_payment_20097,actual_payment_20096,actual_payment_20095,actual_payment_20094,actual_payment_20093,actual_payment_20092,actual_payment_20091,actual_payment_200812,actual_payment_200811,actual_payment_200810,actual_payment_20089,actual_payment_20088,actual_payment_20087,actual_payme

##### Slope calculation

In [64]:
def calculate_slope(row):
    X = np.arange(len(row)).reshape(-1, 1)
    y = row.values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    return model.coef_[0][0]

# Calculate slope for each row
slopes = df.apply(calculate_slope, axis=1)

##### Credit Age

In [76]:
exp_cr_age = exp_acc.groupby('customer_id')['open_dt'].min().reset_index()
exp_cr_age['credit_age_in_months'] = ((observation_date - exp_cr_age['open_dt']) / np.timedelta64(1, 'M'))


##### Months since most recent loan

In [80]:
exp_months_since_recent_tr = exp_acc.groupby('customer_id')['open_dt'].max().reset_index()
exp_months_since_recent_tr['months_since_most_recent_loan'] = ((observation_date - exp_months_since_recent_tr['open_dt']) / np.timedelta64(1, 'M'))

In [87]:
exp_months_since_recent_tr.head()

,customer_id,open_dt,months_since_most_recent_loan
0,6005156530,2022-02-05,23.387097
1,6006456463,2023-09-14,4.483871
2,6200041441,2021-04-01,33.387097
3,6201333763,2023-07-03,6.838710
4,6201413774,2023-04-15,9.387097


##### Months since last payment

In [81]:
exp_months_since_last_pay = exp_acc.groupby('customer_id')['last_payment_dt'].max().reset_index()
exp_months_since_last_pay['months_since_last_payment'] = ((observation_date - exp_months_since_last_pay['last_payment_dt']) / np.timedelta64(1, 'M'))

In [82]:
exp_months_since_last_pay.head()

,customer_id,last_payment_dt,months_since_last_payment
0,6005156530,2023-05-02,8.838710
1,6006456463,NaT,NaN
2,6200041441,2021-05-15,31.967742
3,6201333763,2023-08-01,5.903226
4,6201413774,2023-04-15,9.387097


##### FOIR calculation

In [ ]:
def getEMI(row):
    emi = 0
    if row['total_bal_of_open_pl_l12m']>0:
        emi = emi + max(0,row['total_cl_open_pl_trades_l12m'])*0.035
    if row['total_bal_open_auto_trades_l12m']>0:
        emi = emi + max(0,row['total_cl_open_auto_trades_l12m'])*0.02
    if row['total_balance_open_mortgage_trades_l12m']>0:
        emi = emi + max(0,row['total_cl_open_mortgage_trades_l12m'])*0.012
    if row['no_of_open_cc_trades']>0:
        emi = emi + max(0,row['total_cl_of_cc_trades_l12m']*row['max_aggregate_bankcard_utilisation_l12m']*0.01*0.05)

    return int(emi)

##### Credit Mix

In [116]:
exp_acc[exp_acc.closed_dt.isna() | ((exp_acc.balance_am > 0) & exp_acc.type_trade != 'CC')]

,customer_id,acct_key,account_nb,m_sub_id,acct_type_cd,open_dt,actual_payment_am,asset_class_cd,balance_am,balance_dt,charge_off_am,closed_dt,credit_limit_am,days_past_due,dflt_status_dt,last_payment_dt,orig_loan_am,past_due_am,payment_history_grid,suit_filed_willful_dflt,written_off_and_settled_status,write_off_status_dt,responsibility_cd,portfolio_rating_type_cd,actual_payment_am_01,actual_payment_am_02,actual_payment_am_03,actual_payment_am_04,actual_payment_am_05,actual_payment_am_06,actual_payment_am_07,actual_payment_am_08,actual_payment_am_09,actual_payment_am_10,actual_payment_am_11,actual_payment_am_12,actual_payment_am_13,actual_payment_am_14,actual_payment_am_15,actual_payment_am_16,actual_payment_am_17,actual_payment_am_18,actual_payment_am_19,actual_payment_am_20,actual_payment_am_21,actual_payment_am_22,actual_payment_am_23,actual_payment_am_24,actual_payment_am_25,actual_payment_am_26,actual_payment_am_27,actual_payment_am_28,actual_payment_am_29,actual_payment_am_30,actual_payment_am_31,actual_payment_am_32,actual_payment_am_33,actual_payment_am_34,actual_payment_am_35,actual_payment_am_36,balance_am_01,balance_am_02,balance_am_03,balance_am_04,balance_am_05,balance_am_06,balance_am_07,balance_am_08,balance_am_09,balance_am_10,balance_am_11,balance_am_12,balance_am_13,balance_am_14,balance_am_15,balance_am_16,balance_am_17,balance_am_18,balance_am_19,balance_am_20,balance_am_21,balance_am_22,balance_am_23,balance_am_24,balance_am_25,balance_am_26,balance_am_27,balance_am_28,balance_am_29,balance_am_30,balance_am_31,balance_am_32,balance_am_33,balance_am_34,balance_am_35,balance_am_36,credit_limit_am_01,credit_limit_am_02,credit_limit_am_03,credit_limit_am_04,credit_limit_am_05,credit_limit_am_06,credit_limit_am_07,credit_limit_am_08,credit_limit_am_09,credit_limit_am_10,credit_limit_am_11,credit_limit_am_12,credit_limit_am_13,credit_limit_am_14,credit_limit_am_15,credit_limit_am_16,credit_limit_am_17,credit_limit_am_18,credit_limit_am_19,credit_limit_am_20,credit_limit_am_21,credit_limit_am_22,credit_limit_am_23,credit_limit_am_24,credit_limit_am_25,credit_limit_am_26,credit_limit_am_27,credit_limit_am_28,credit_limit_am_29,credit_limit_am_30,credit_limit_am_31,credit_limit_am_32,credit_limit_am_33,credit_limit_am_34,credit_limit_am_35,credit_limit_am_36,past_due_am_01,past_due_am_02,past_due_am_03,past_due_am_04,past_due_am_05,past_due_am_06,past_due_am_07,past_due_am_08,past_due_am_09,past_due_am_10,past_due_am_11,past_due_am_12,past_due_am_13,past_due_am_14,past_due_am_15,past_due_am_16,past_due_am_17,past_due_am_18,past_due_am_19,past_due_am_20,past_due_am_21,past_due_am_22,past_due_am_23,past_due_am_24,past_due_am_25,past_due_am_26,past_due_am_27,past_due_am_28,past_due_am_29,past_due_am_30,past_due_am_31,past_due_am_32,past_due_am_33,past_due_am_34,past_due_am_35,past_due_am_36,payment_rating_cd_01,payment_rating_cd_02,payment_rating_cd_03,payment_rating_cd_04,payment_rating_cd_05,payment_rating_cd_06,payment_rating_cd_07,payment_rating_cd_08,payment_rating_cd_09,payment_rating_cd_10,payment_rating_cd_11,payment_rating_cd_12,payment_rating_cd_13,payment_rating_cd_14,payment_rating_cd_15,payment_rating_cd_16,payment_rating_cd_17,payment_rating_cd_18,payment_rating_cd_19,payment_rating_cd_20,payment_rating_cd_21,payment_rating_cd_22,payment_rating_cd_23,payment_rating_cd_24,payment_rating_cd_25,payment_rating_cd_26,payment_rating_cd_27,payment_rating_cd_28,payment_rating_cd_29,payment_rating_cd_30,payment_rating_cd_31,payment_rating_cd_32,payment_rating_cd_33,payment_rating_cd_34,payment_rating_cd_35,payment_rating_cd_36,days_past_due_01,days_past_due_02,days_past_due_03,days_past_due_04,days_past_due_05,days_past_due_06,days_past_due_07,days_past_due_08,days_past_due_09,days_past_due_10,days_past_due_11,days_past_due_12,days_past_due_13,days_past_due_14,days_past_due_15,days_past_due_16,days_past_due_17,days_past_due_18,days_past_due_19,days_past_due_20,days_past_due_21,days_past_due_22,days_past_due_23,da

In [115]:
exp_acc.type_trade.value_counts()

type_trade
PL        937963
CC        186302
AT         55066
Others     32544
BL         26220
MT         21794
CL           396
Name: count, dtype: int64